# Desenvolvimento da Fato Vendas

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [2]:
 # Importando dados 

import pandas as pd 
from sqlalchemy import text
from sqlalchemy import create_engine
from dotenv import load_dotenv
from utils.acessobanco import acessobanco

In [3]:
# Carrega as variáveis de ambiente do arquivo .env

load_dotenv(dotenv_path="/home/danielpedro/Python/Projetos/SuperStorePY/Bancos/acessobanco.env")


True

In [4]:
# Cria a conexão com o banco usando a função acessobanco()

engine = acessobanco()

In [5]:
# criando o dataframe atraves da query com left join

df_fato = pd.read_sql("""select
	id as sk_pedido
	,calenPed.sk_data  as sk_data_pedido
	,calenEnv.sk_data  as sk_data_envio
	,mte.sk_modo_envio 
	,c.sk_cliente 
	,v.sk_vendedores 
	,p.sk_produto 
	,valor_venda
	,quantidade
	,desconto
	,lucro
	 from stg.orders as ord
	 left join dw.dim_metodo_envio_scd1 as mte
	 on ord.modo_envio  = mte.modo_envio	 
	 left join dw.dim_cliente_scd1 as c
	 on ord.id_cliente  = c.id_cliente	 
	 left join dw.dim_produto_scd2 as p
	 on ord.id_produto = p.id_produto 
	 left join dw.dim_vendedores_scd1 v
	 on ord.regiao = v.regiao
	 left join dw.dim_calendario AS calenPed
     on TO_DATE(ord.data_pedido, 'YYYY-MM-DD') = calenPed.data
	 left join dw.dim_calendario AS calenEnv
     on TO_DATE(ord.data_envio, 'YYYY-MM-DD') = calenEnv.data""",engine)

In [6]:
# Realizando um top 5 dos dados

df_fato.head()

,sk_pedido,sk_data_pedido,sk_data_envio,sk_modo_envio,sk_cliente,sk_vendedores,sk_produto,valor_venda,quantidade,desconto,lucro
0,646,20171230,20180105,1,161,3,615,209.3,2,0.0,56.510999999999996
1,907,20171230,20180103,1,590,2,388,323.136,4,0.2,12.117599999999968
2,908,20171230,20180103,1,590,2,732,90.93,7,0.0,2.7278999999999964
3,909,20171230,20180103,1,590,2,1658,52.775999999999996,3,0.2,19.791
4,1297,20171230,20180103,1,264,1,412,13.904,2,0.2,4.5188


In [7]:
# Verificando os tipos das colunas

df_fato.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   sk_pedido       9994 non-null   object
 1   sk_data_pedido  9994 non-null   int64 
 2   sk_data_envio   9994 non-null   int64 
 3   sk_modo_envio   9994 non-null   int64 
 4   sk_cliente      9994 non-null   int64 
 5   sk_vendedores   9994 non-null   int64 
 6   sk_produto      9994 non-null   int64 
 7   valor_venda     9994 non-null   object
 8   quantidade      9994 non-null   object
 9   desconto        9994 non-null   object
 10  lucro           9994 non-null   object
dtypes: int64(6), object(5)
memory usage: 859.0+ KB


In [8]:
# Criando um truncate 

tabela = 'fato_vendas'
schema = 'dw' 

try:
    with engine.begin() as conn:
        print(f'Limpando tabela {schema}.{tabela}...')
        conn.execute(text(f'TRUNCATE TABLE {schema}.{tabela}'))
        print(f'Tabela {schema}.{tabela} truncada com sucesso!')
        
except Exception as e:
    print(f'Erro ao truncar a tabela {schema}.{tabela}: {e}')
    print('Detalhes do erro:', str(e))

Limpando tabela dw.fato_vendas...
Tabela dw.fato_vendas truncada com sucesso!


In [9]:
# Importando os dados para a tabela

with engine.begin() as conn:  
    df_fato.to_sql(
        "fato_vendas",      # nome da tabela
        conn,               # conexão com o banco de dados
        schema="dw",        # schema onde a tabela será criada
        if_exists="append", # substitui a tabela se já existir (garante dados atualizados)
        index=False,         # não cria coluna de índice do pandas
        method="multi",      #insere várias linhas por comando INSERT (melhor performance)
        chunksize=5000       #envia os dados em blocos de 5000 linhas (controle de memória)
    ) 

In [10]:
# Verificando a quantidade de linhas dentro do banco

count = pd.read_sql("SELECT COUNT(*) as total_registros FROM dw.fato_vendas", engine).iloc[0,0]
print(f"Total de registros na tabela: {count}")

Total de registros na tabela: 9994
